In [1]:
import numpy as np
import pandas as pd
import time

import requests                     #This is for scraping
from bs4 import BeautifulSoup       #This is for scraping
import webbrowser                   #Scraping web
import csv

In [2]:
import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import io 
from selenium.webdriver.common.keys import Keys

In [3]:
#conda install -c anaconda xlrd
#Installs stuff for excel

In [4]:
#This is the database that I want to upgrade with the scraped content
df = pd.read_excel("data.xlsx",squeeze=True, encoding ="utf-8",dtype=str)
index = df.index
values = df.values

In [5]:
df.head()

,id,electoral_cycle,source,name,sex,gov_support,exact_seat,committee_membership,party_hierarchy,seniority,...,cm_from_year,cm_to_day,cm_to_month,cm_to_year,ph_from_day,ph_from_month,ph_from_year,ph_to_day,ph_to_month,ph_to_year
0,a182_1990_003_19921020_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,abraham_tibor,m,0,pest,3,99995,1,...,1992,27,6,1994,99995,99995,99995,99995,99995,99995
1,a302_1990_001_19900503_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,ader_janos,m,0,fidesz,1,99995,1,...,1990,24,3,1992,99995,99995,99995,99995,99995,99995
2,a302_1990_009_19920324_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,ader_janos,m,0,fidesz,9,99995,1,...,1992,8,4,1992,99995,99995,99995,99995,99995,99995


In [6]:
#selenium login start
#It seems like we need selenium, since we got a dynamic webpage... unlucky
driver = webdriver.Chrome('./chromedriver')
driver.get('https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=MbCEKk7P&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_lis%3Fp_ckl%3D40')
html_source = driver.page_source #Makes soup
soup = BeautifulSoup(html_source, 'html.parser')
print(soup.prettify())

<html class="ltr yui3-js-enabled js flexbox canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface no-generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths webkit js chrome chrome85 chrome85-0 win secure no-high-contrast-mode" dir="ltr" lang="hu-HU" style="">
 <div class="" id="yui3-css-stamp" style="position: absolute !important; visibility: hidden !important">
 </div>
 <head>
  <title>
   Képviselők ciklusonkénti adatai - Országgyűlés
  </title>
  <meta content="initial-scale=1.0, width=device-width" name="viewport"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <script async="" src="//www.google-analytics.com/analytics.js">
  </

In [7]:
#Let's see, what kind of links does the page contain. The goal is to click on the links of the parliament members.
#First, let's make the list of all the urls. 
url_list = []
for url in soup.find_all('a'):
    url_list.append(url)

#Now, let's take a look on our data. 
for i in range (len(url_list)):
    print(i)
    print(url_list[i])


0
<a href="#main-content">Ugrás a tartalomhoz</a>
1
<a href="https://www.parlament.hu/web/guest/orszaggyules">
<span>ORSZÁGGYŰLÉS</span>
</a>
2
<a href="https://www.parlament.hu/web/guest/bevezetes-jogi-szabalyzas1" role="menuitem">
<span>Bevezetés, jogi szabályozás</span>
</a>
3
<a href="https://www.parlament.hu/web/guest/mukodes-feladatok1" role="menuitem">
<span>Működés, feladatok</span>
</a>
4
<a href="https://www.parlament.hu/web/guest/nemzetpolitika" role="menuitem">
<span>Nemzetpolitika</span>
</a>
5
<a href="https://www.parlament.hu/web/guest/europai-unios-tevekenyseg" role="menuitem">
<span>Európai uniós tevékenység</span>
</a>
6
<a href="https://www.parlament.hu/web/guest/interparlamentaris-unio1" role="menuitem">
<span>Az Interparlamentáris Unió</span>
</a>
7
<a href="https://www.parlament.hu/web/guest/az-orszaggyules-barati-tagozatai" role="menuitem">
<span>Az Országgyűlés baráti tagozatai</span>
</a>
8
<a href="https://www.parlament.hu/web/orszaghaz">
<span>Országház</span

In [8]:
#At Access Date: 19/09/2020, the parliamentary members from A to Z are Ander Balázs to Zsigó Róbert. 
#Ander Balázs has a link at 231 number, Zsigó has at 464, so I will only keep these elements in the url_list. 
#It's important, that this might change in the future, but right now, my goal is to find and click on the links with the names. 
url_list = url_list[231:464]
print(url_list)

[<a class="" href="https://www.parlament.hu:443/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&amp;p_p_lifecycle=1&amp;p_p_state=normal&amp;p_p_mode=view&amp;p_auth=X8SB2QCu&amp;_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da716%26p_ckl%3D40%26p_osszefuz%3D">Ander Balázs</a>, <a class="" href="https://www.parlament.hu:443/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&amp;p_p_lifecycle=1&amp;p_p_state=normal&amp;p_p_mode=view&amp;p_auth=X8SB2QCu&amp;_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da018%26p_ckl%3D40%26p_osszefuz%3D">Dr. Apáti István</a>, <a class="" href="https://www.parlament.hu:443/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9

In [9]:
#I convert the url to string
for i in range (len(url_list)):
    url_list[i] = str(url_list[i])

In [10]:
#The href links won't direct me to the parliament member's page with request, I have to click on it. 
#All of the href elements has a name tag. I will collect all the names of the parliament members from the page. 
Names = []
for i in range(len(url_list)):
    if len(url_list[i]) > 400:
        name = (url_list[i][405:(len(url_list[i])-4)])
        Names.append(name)
#So, what do we have? 
print(Names)

['Ander Balázs', 'Dr. Apáti István', 'Dr. Aradszki András', 'Ágh Péter', 'B. Nagy László', 'Bajnai Gordon', 'Balczó Zoltán', 'Balla György', 'Balla Mihály', 'Balog Zoltán', 'Bana Tibor', 'Bangóné Borbély Ildikó', 'Bánki Erik', 'Bányai Gábor', 'Dr. Bárándy Gergely', 'Bartos Mónika', 'Becsó Zsolt', 'Bencsik János', 'Dr. Bene Ildikó', 'Bíró Márk', 'Bodó Sándor', 'Boldog István', 'Bóna Zoltán', 'Dr. Botka László', 'Budai Gyula', 'Burány Sándor', 'Csenger-Zalán Zsolt', 'Cseresnyés Péter', 'Csizi Péter', 'Csöbör Katalin', 'Czerván György', 'Dr. Czomba Sándor', 'Czunyiné Dr. Bertalan Judit', 'Dankó Béla', 'Demeter Márta', 'Demeter Zoltán', 'Dunai Mónika', 'Dúró Dóra', 'Egyed Zsolt', 'Farkas Flórián', 'Farkas Gergely', 'Farkas Sándor', 'Dr. Fazekas Sándor', 'Firtl Mátyás', 'Fodor Gábor', 'Dr. Fónagy János', 'Font Sándor', 'Földi László', 'Gaal Gergely', 'Dr. Galambos Dénes', 'Gelencsér Attila', 'Gőgös Zoltán', 'Gulyás Gergely', 'Gúr Nándor', 'Gyopáros Alpár', 'Gyöngyösi Márton', 'Győrffy Baláz

In [11]:
#Gives the party membership position of the person between 2014 and 2018
def get_party(soup):
    trigger = 0
    for element in soup.find_all('td'):
        element = str(element)
        if trigger == 1:
            result = element[4:-5]
            trigger = 2
        if element == "<td>2014-2018</td>" and trigger == 0:
            trigger = 1
    return result

In [12]:
#It is time for the scrape loop, I click on the links with the Parliamentary members. 
Party = []
Source_url = [] 

for i in range(len(Names)):
    driver.get('https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=MbCEKk7P&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_lis%3Fp_ckl%3D40')
    #Next line: clicks on the element on the page, that contains the matching name. 
    link = driver.find_element_by_link_text(Names[i])
    link.click()
    Source_url.append(driver.current_url)
    html_source = driver.page_source #Makes soup
    soup = BeautifulSoup(html_source, 'html.parser')
    target = get_party(soup)
    Party.append(target)
    
    print(Names[i])
    print(driver.current_url)
    print(target)
    #Sleep a bit, dude! 
    time.sleep(1)   

Ander Balázs
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da716%26p_ckl%3D40%26p_osszefuz%3D
Jobbik
Dr. Apáti István
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da018%26p_ckl%3D40%26p_osszefuz%3D
Jobbik
Dr. Aradszki András
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlam

Bóna Zoltán
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db764%26p_ckl%3D40%26p_osszefuz%3D
Fidesz
Dr. Botka László
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db164%26p_ckl%3D40%26p_osszefuz%3D
MSZP
Budai Gyula
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pai

Fodor Gábor
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Df319%26p_ckl%3D40%26p_osszefuz%3D
független
Dr. Fónagy János
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Df006%26p_ckl%3D40%26p_osszefuz%3D
Fidesz
Font Sándor
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cm

Hegedűs Lorántné
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dh057%26p_ckl%3D40%26p_osszefuz%3D
Jobbik
Hende Csaba
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dh021%26p_ckl%3D40%26p_osszefuz%3D
Fidesz
Heringes Anita
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cm

Kontrát Károly
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dk029%26p_ckl%3D40%26p_osszefuz%3D
Fidesz
Kónya Péter
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dk763%26p_ckl%3D40%26p_osszefuz%3D
független
Korózs Lajos
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms

Manninger Jenő
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dm014%26p_ckl%3D40%26p_osszefuz%3D
Fidesz
Mátrai Márta
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dm006%26p_ckl%3D40%26p_osszefuz%3D
Fidesz
Dr. Mengyi Roland
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_

Pócs János
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dp041%26p_ckl%3D40%26p_osszefuz%3D
Fidesz
Pogácsás Tibor
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dp006%26p_ckl%3D40%26p_osszefuz%3D
Fidesz
Dr. Pósán László
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms

Simonka György
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Ds106%26p_ckl%3D40%26p_osszefuz%3D
Fidesz
Sneider Tamás
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Ds107%26p_ckl%3D40%26p_osszefuz%3D
Jobbik
Soltész Miklós
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cm

Tessely Zoltán
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dt038%26p_ckl%3D40%26p_osszefuz%3D
Fidesz
Dr. Tiba István
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dt017%26p_ckl%3D40%26p_osszefuz%3D
Fidesz
Tiffán Zsolt
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cm

Vécsey László
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv044%26p_ckl%3D40%26p_osszefuz%3D
Fidesz
Dr. Vejkey Imre
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv055%26p_ckl%3D40%26p_osszefuz%3D
KDNP
Velez Árpád
https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=X8SB2QCu&_hu_parlament_cms_pa

In [13]:
#I am decoding Party membership to a dummy variable, where 1 relates to government parties, 0 relates to opposition parties
gov_support = []
for element in Party:
    if element == "Fidesz" or element == "KDNP":
        gov_support.append(1)
    else:
        gov_support.append(0)
print(gov_support)

[0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1]


In [14]:
#Put data into the dataframe
for i in range(len(Names)):
    new_row = {'name':Names[i], 'gov_support':gov_support[i], 'source':Source_url[i]}
    df = df.append(new_row, ignore_index=True)


In [17]:
df

,id,electoral_cycle,source,name,sex,gov_support,exact_seat,committee_membership,party_hierarchy,seniority,...,cm_from_year,cm_to_day,cm_to_month,cm_to_year,ph_from_day,ph_from_month,ph_from_year,ph_to_day,ph_to_month,ph_to_year
0,a182_1990_003_19921020_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,abraham_tibor,m,0,pest,3,99995,1,...,1992,27,6,1994,99995,99995,99995,99995,99995,99995
1,a302_1990_001_19900503_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,ader_janos,m,0,fidesz,1,99995,1,...,1990,24,3,1992,99995,99995,99995,99995,99995,99995
2,a302_1990_009_19920324_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,ader_janos,m,0,fidesz,9,99995,1,...,1992,8,4,1992,99995,99995,99995,99995,99995,99995
3,NaN,NaN,https://www.parlament.hu/web/guest/kepviselok-...,Ander Balázs,NaN,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,https://www.parlament.hu/web/guest/kepviselok-...,Dr. Apáti István,NaN,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,NaN,NaN,https://www.parlament.hu/web/guest/kepviselok-...,Dr. Völner Pál,NaN,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,NaN,NaN,https://www.parlament.hu/web/guest/kepviselok-...,Witzmann Mihály,NaN,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211,NaN,NaN,https://www.parlament.hu/web/guest/kepviselok-...,Z. Kárpát Dániel,NaN,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,NaN,NaN,https://www.parlament.hu/web/guest/kepviselok-...,Dr. Zombor Gábor,NaN,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
driver.get('https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=MbCEKk7P&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_lis%3Fp_ckl%3D40')
link = driver.find_element_by_link_text("Hollik István")
link.click()

In [ ]:
html_source = driver.page_source #Makes soup
soup = BeautifulSoup(html_source, 'html.parser')
print(soup.prettify())

In [ ]:
for element in soup.find_all('td'):
    element = str(element)
    print(element)